# 3조 News 데이터 활용

In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
import sys
import json
from konlpy.tag import Okt
from collections import Counter
import numpy as np
import matplotlib
from IPython.display import set_matplotlib_formats
from PIL import Image
import os
import pandas as pd
from datetime import datetime

# SBS

In [2]:
def get_sbs(date):
    
    base_url = 'https://news.sbs.co.kr/news/newsSection.do'

    params = {
        'sectionType' : '01',
        'plink' : 'SNB',
        'cooper' : 'SBSNEWS',
        'pageIdx' : 1,
        'pageDate' : date
    }

    index_list = ['01', '02', '03', '07', '08', '14', '09'] # SectionType ( 카테고리를 가져오기 위한 리스트)

    index_str_list = ['정치', '경제', '사회', '국제', '생활&문화', '연예', '스포츠'] # SectionType = 카테고리 이름

    txt_file_name = ['a1.txt', 'b1.txt', 'c1.txt', 'd1.txt', 'e1.txt', 'f1.txt', 'g1.txt'] #각 카테고리별 저장을 위한 text 파일 list

    
    index_point = 0 # 리스트를 가져 오기 위한 index
    
    f = open(txt_file_name[index_point], "w+t") # 무조건 처음 시작할 때 지우고 다시만든다.
    

    while True:
        resp = requests.get(base_url, params = params)
        soup = BeautifulSoup(resp.text, 'lxml')
        news_list_tags = soup.select('#container > div > div.w_news_list.type_issue > ul > li > a.news')

        if(len(news_list_tags) == 0): # 더이상 탐색할 페이지가 없을때

            if(index_point + 1 == 7): # section 및 page 다 탐색 -> 종료
                f.close()
                break

            else:
                f.close()
                index_point += 1
                f = open(txt_file_name[index_point], "w+t")
                
                params['sectionType'] = index_list[index_point]
                params['pageIdx'] = 1

                continue
            break

        for i in news_list_tags: # 파일을 쓴다.

            dt_tags = i.find_all('strong')
            title_tag = dt_tags[0].text
            f.write(title_tag)
            f.write("\n")

        params['pageIdx'] += 1

    f.close()

# JTBC

In [3]:
def jtbc_news_list(day):
    
    # 카테고리 'scode'를 dict로 정의.
    menu_list = {
        '속보' : 0,
        '정치' : 10,
        '경제' : 20,
        '사회' : 30,
        '국제' : 40,
        '문화' : 50,
        '연예' : 60,
        '스포츠' : 70,
        '날씨': 80
    }
    
    base_url ='http://news.jtbc.joins.com/section/list.aspx?'
    params = {
        'scode' : 0,
        'pdate' : day,
        'pgi' : 0
    }
    
    # text 파일 통합 위해서 이름명 설정. 날씨 카테고리 문화에 합침.
    scode_list = {
        10 : 'a',
        20 : 'b',
        30 : 'c',
        40 : 'd',
        50 : 'e',
        60 : 'f',
        70 : 'g',
        80 : 'e'
    }
    
    for k in range(8):
        news_list = []
        params['scode'] +=10 # 카테고리 넘기기
        params['pgi'] = 0
        
        while True:
            params['pgi'] +=1 # 페이지 넘기기

            resp = requests.get(base_url, params=params)
            soup = BeautifulSoup(resp.text, 'lxml')

            news_tags = soup.find('ul', id='section_list')
            news_list_tags = news_tags.find_all('dt', class_='title_cr') # 뉴스 타이틀 tag


            if len(news_list_tags)==0: # 마지막 페이지에서 더 넘어가면 while문 탈출.
                break

            for tags in news_list_tags:
                news_title = tags.text

                news_list.append(
                    news_title.strip()
                )
        
    
        file_name = scode_list[params['scode']]+'2.txt' # 카테고리별 params 숫자를 받아 파일명 key값으로 입력 -> 알파벳 출력
        
        
        # 날씨 카테고리 내용 문화영역에 추가하기. 나머지는 그냥 파일 생성.
        if params['scode'] == 80:
            file1 = open(file_name, 'at')
        else:
            file1 = open(file_name, 'w')
        
        for i in news_list:
            file1.write(i+"\n")
            
        file1.close()

# YTN

In [4]:
import re

def make_f(newlist):
    # 파일명
    menu_list=['a3.txt','c3.txt','b3.txt',
          'd3.txt','e3.txt','g3.txt']
    for idx, i in enumerate(menu_list):
        f = open(i,'w')
        for j in newlist[idx]:
            msg = j
            f.write(j+"\n")
        f.close()
        #main()    

In [5]:
#스포츠 메뉴의 구조가 달라 따로 구함
def sport(Sports_list, input_day):

    for i in range(3):
        params={
            'page':i+1,
            's_mcd':'0107'
        }
        resp = requests.get('https://www.ytn.co.kr/photo/photo_list.php',params=params)
        soup = BeautifulSoup(resp.text, 'lxml')
        #리스트
        sec_tag = soup.find("div", id="ytn_list_v2014")
        dl_tag=sec_tag.find_all("dl",class_='photo_list')
        
        for j in dl_tag:
            # 날짜
            date_tag = j.find('dd',class_='date')
            date_d = date_tag.text
            date = date_d.split(" ")
            date = date[0].replace("[","")


            if input_day == date:
                dt_tag =j.find_all('dt')
                a_tag = dt_tag[0].find("a")
                title = a_tag.text
                
                
                if title in Sports_list:
                    continue
                    
                Sports_list.append(title)
                
      

In [10]:
def get_ytn(i_date):
    
    year = str(i_date)[:4]
    month = str(i_date)[4:6]
    day = str(i_date)[6:]
    input_day = year + "-" + month + "-" + day
    
    # YTN사이트
    url = 'https://www.ytn.co.kr/news/news_list_0101.html'

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    menu = soup.find("ul",id='sub_1')
    li_tag = menu.find_all('li',recursive=False)

    a=[]
    politics_list=[]
    economy_list=[]
    social_list=[]
    domestic_list=[]
    International_list=[]
    Science_list=[]
    Sports_list=[]
    culture_list =[]


    # 카테고리마다 뉴스 제목 리스트
    newlist = [politics_list,economy_list,social_list,domestic_list,
               International_list,Science_list,culture_list]


    #3,4,5,6,7,8,9,10,11
    url2 = 'https://www.ytn.co.kr/news/news_list.php?'
    for idx in range(3,11,1):
        #메뉴이동
        a_tag =li_tag[idx].find('a')
        move = a_tag.get('href')
        a = re.find_all('\d+',move)

        if idx == 10:
            sport(Sports_list, input_day)
            newlist.remove(domestic_list)
            newlist.remove(Science_list)
            newlist.append(Sports_list)
            make_f(newlist)
            break


        for i in range(3):
            params={
                'page':i+1,
                's_mcd':a[0]
            }
            resp = requests.get(url2,params=params)
            soup = BeautifulSoup(resp.text, 'lxml')

            #리스트
            sec_tag = soup.find("div", id="ytn_list_v2014")

            span_tag=sec_tag.find_all("dl", class_="news_list_v2014")

            for j in span_tag:
                # 날짜
                date_tag = j.find('dd',class_='date')
                date_d = date_tag.text
                date = date_d.split(" ")
                date = date[0].replace("[","")


                if input_day == date:
                    dt_tag =j.find_all('dt')
                    a_tag = dt_tag[0].find("a")
                    title = a_tag.text


                    if title in newlist[idx-3]:
                        continue
                    if idx == 6 or idx == 8:
                        newlist[6].append(title)

                    else:    
                        newlist[idx-3].append(title)



# MBN

In [11]:
############################################################
##########프로젝트 MBN 홈페이지 긁어오는 함수 코드##########
############################################################

def MBN_News_Function(i_date):
    
    input_date = str(i_date)
    
    count=0
    
    reresult_list=[]
    while count<7:
        
        if count==0:
            category_s='politics'
            MBN_url='https://www.mbn.co.kr/news/politics/'
        
        if count==1:
            category_s='economy'
            MBN_url='https://www.mbn.co.kr/news/economy/'

        elif count==2:
            category_s='society'
            MBN_url='https://www.mbn.co.kr/news/society/'

        elif count==3:
            category_s='world'
            MBN_url='https://www.mbn.co.kr/news/world/'

        elif count==4:
            category_s='culture'
            MBN_url='https://www.mbn.co.kr/news/culture/'

        elif count==5:
            category_s='entertain'
            MBN_url='https://www.mbn.co.kr/news/entertain/'        

        elif count==6:
            category_s='sports'
            MBN_url='https://www.mbn.co.kr/news/sports/'

        result_list=[]
        for i in range(1,8): #페이지번호
            params={
                'page':i,
                'vod':None,
                'category':category_s
            }
            resp=requests.get(MBN_url, params=params)

            resp.encoding='eur-kr'
            
            soup=BeautifulSoup(resp.text, 'lxml')

            MBN_news_list=soup.select('div.list_area > dl.article_list > dt.tit > a')
            MBN_news_list_date=soup.select('div.list_area > dl.article_list > dd.desc > span.date')

            news_list=[]
            for news,date in zip(MBN_news_list, MBN_news_list_date):

                news=news.text.strip() #제목 내 띄어쓰기 제거
                news=news.replace("\'" , " ") #제목 내 따옴표 제거
                news=news.replace("♥" , " ") #제목 내 따옴표 제거
                news=news.replace("[포토]"," ") #제목 내 [포토] 제거

                news_day=date.text[0:10].replace("-","")

                if news_day == input_date: #날짜 수정하면 됨
                    news_list.append(news)
            result_list.extend(news_list)
        
        count+=1
        
        
        reresult_list.append(result_list)
    return reresult_list



############################################################
#########프로젝트 MBN 메모장으로 꺼내오는 함수 코드#########
############################################################


def mbn(day):

    title_list=['a','b','c','d','e','f','g']
    #나중에 j를 e로 바꿔서 추가해야함!

    f_list=[]
    for title in title_list:
        f=open('{}4.txt'.format(title),'w', -1, 'utf-8')    
        f_list.append(f)

        
    MBN=MBN_News_Function(day)   # 2중리스트로 옴. [[1,2,3...] , [1,2,3...] , [1,2,3...]]


    for i, f in zip(MBN,f_list):
        for j in i:
            f.write(j)
            f.write('\n')
        f.close()

# 아시아경제

In [12]:
def asia(date):
    
    count=0
    
    while count<6:
        day = int(datetime.today().strftime("%Y%m%d")) - int(date)
        
        if count==0:
            ASIA_url='https://www.asiae.co.kr/list/politics-all'
            
        if count==1:
            ASIA_url='https://www.asiae.co.kr/list/economy-all'

        elif count==2:
            ASIA_url='https://www.asiae.co.kr/list/society-all'

        elif count==3:
            ASIA_url='https://www.asiae.co.kr/list/world-all'

        elif count==4:
            ASIA_url='https://www.asiae.co.kr/list/life-all'

        elif count==5:
            ASIA_url='https://www.asiae.co.kr/list/entertainment-sports-all'
        
        page_num = 1
        check = 0
        result_list = []
        
        while True: #페이지번호
            Ex_url = ASIA_url + '/' + str(page_num)
            resp=requests.get(Ex_url) # 1페이지부터 탐색
            resp.encoding='eur-kr'
            soup=BeautifulSoup(resp.text, 'lxml')
            ASIA_news_list = soup.select('div.cont_listarea > div > div.listsm_type > h3.l_fsttit > a')
            ASIA_news_time = soup.select('div.cont_listarea > div > div.listsm_type > span')
            
            
            for li, time in zip(ASIA_news_list, ASIA_news_time):
                
                if(time.text == str(day + 1) +'일 전'):
                    check += 1
                    break
                elif(time.text == str(day) + '일 전'):
                    result_list.append(li.text.strip())

            if(check == 1): 
                break
                    
            page_num += 1 # Page 넘기기
            
        
        
        title_list=['a','b','c','d','e','f']
        
        file_name = title_list[count] + '5.txt'
        f = open(file_name, "w+t") # 무조건 처음 시작할 때 지우고 다시만든다.
        for i in result_list:
            f.write(i)
            f.write('\n')
        f.close()
        
        count+=1


# 메인

In [13]:
date = 20200924

get_sbs(date)
jtbc_news_list(date)
get_ytn(date)
mbn(date)
asia(date)

# # ################################# 텍스트에 Data 저장된 상태
txt_save_list = ['정치.txt', '경제.txt', '사회.txt', '국제.txt', '생활&문화.txt', '연예.txt', '스포츠.txt']

txt_file_list = [
    'a1.txt', 'a2.txt', 'a3.txt', 'a4.txt', 'a5.txt',
    'b1.txt', 'b2.txt', 'b3.txt', 'b4.txt', 'b5.txt',
    'c1.txt', 'c2.txt', 'c3.txt', 'c4.txt', 'c5.txt',
    'd1.txt', 'd2.txt', 'd3.txt', 'd4.txt', 'd5.txt',
    'e1.txt', 'e2.txt', 'e3.txt', 'e4.txt', 'e5.txt',
    'f1.txt', 'f2.txt', 'f3.txt', 'f4.txt', 'f5.txt',
    'g1.txt', 'g2.txt', 'g3.txt', 'g4.txt', 'g5.txt' 
]

title_name_list=['정치','경제','사회','국제','생활&문화','연예','스포츠']


category_index_point = 0


while category_index_point < 7: # 카테고리 탐색
    
    SBS = []    
    JTBC= []    
    YTN = []   
    MBN= []     
    ASIA = []   
                                                                      #txt_file_list  index 들어가는 값들
    f = open(txt_file_list[category_index_point * 5] , 'r')           # 1번 파일: 0, 5, 10, 15, 20, 25, 30
    SBS = f.read()
    f.close()
    
    f = open(txt_file_list[(category_index_point*5) + 1] , 'r')       # 2번 파일: 1, 6, 11, 16, 21, 26, 31
    JTBC = f.read()
    f.close()
    
    if((category_index_point * 5) + 2) == 27:                         # idex=27인 f3파일 없음.
        pass
    else:
        f = open(txt_file_list[(category_index_point*5) + 2] , 'r')   # 3번 파일: 2, 7, 12, 17, 22,  , 32
        YTN = f.read()
        f.close()
    
    f = open(txt_file_list[(category_index_point*5) + 3] , 'r', encoding='utf-8')  # 4번 파일: 3, 8, 13, 18, 23, 28, 33
    MBN = f.read()
    f.close()
    
    if((category_index_point * 5) + 4) == 34:
        pass
    else:
        f = open(txt_file_list[(category_index_point*5) + 4] , 'r')   # 5번 파일: 4, 9, 14, 19, 24, 29, 34
        ASIA = f.read()
        f.close()
    
    
    # 카테고리별로 뉴스사 데이터 합치기.
    Category_sum = []
    
    if((category_index_point * 5) + 2) != 27 and ((category_index_point * 5) + 4) != 34:
        Category_sum = SBS + JTBC + YTN + MBN + ASIA
        
    elif ((category_index_point * 5) + 2) == 27:
        Category_sum = SBS + JTBC + MBN + ASIA
    
    elif ((category_index_point * 5) + 4) == 34:
        Category_sum = SBS + JTBC + YTN + MBN
    

    f = open(txt_save_list[category_index_point], 'w+t')
    f.write(Category_sum)
    f.close()
    
    
    # 뒤에 전체 키워드 뽑기위해서 따로 total.txt 생성
    if os.path.exists('total.txt'): # total.txt 가 존재하면 내용추가(a)
        f1 = open('total.txt', 'a')
        f1.write(Category_sum)
        f1.close()
    else:
        f1 = open('total.txt', 'w') # total.txt가 없으면 새로 생성해서 쓰기.
        f1.write(Category_sum)
        f1.close()
    
    
    
    category_index_point += 1 # 다음 카테고리

    
    #### 순위를 찾아내기위한 문장 리스트###
category_index_point = 0

while category_index_point < 7:

    line_list=[]
    
    f = open(txt_save_list[category_index_point], 'r')
    lines = f.read()
    line_list = lines.split('\n')
    
    
    nlpy = Okt()
    nouns = nlpy.nouns(lines)
    count = Counter(nouns)

    tag_count = []
    
    for n, c in count.most_common(100):
        dics = {'키워드' : n, '중복수' : c}
        if len(dics['키워드']) >= 2 and len(tag_count) <= 100:
            tag_count.append(dics)
            

    #### 순위 3위까지의 리스트#####
    rank=[]

    
    print('---------------카테고리[{}]--------------------'.format(title_name_list[category_index_point]) )
    for idx,tag in enumerate(tag_count):    
     #### 순위 3위까지의 리스트 뽑아내기 && 기사 뽑아내기#####
        if(idx == 0 or idx == 1 or idx == 2 ):
            rank.append(tag['키워드'])   
    
    print(pd.DataFrame(tag_count[:20], index = list(range(1,21)) ))

            
    print("\n ********** 순위 ************* \n")  
    print('\t 1순위:{}'.format(rank[0]))
    print('\t 2순위:{}'.format(rank[1]))
    print('\t 3순위:{}'.format(rank[2]))  
    
    rank_li=[]
    
    temp=0
    for line_li in line_list:
        if rank[0] in line_li:
            rank_li.append(line_li+'\n')
            break
            

    temp=0
    for line_li in line_list:
        if rank[1] in line_li:
            rank_li.append(line_li+'\n')
            break
            
            
    temp=0
    for line_li in line_list:
        if rank[2] in line_li:
            rank_li.append(line_li+'\n') 
            break
             
                
    print("\n ********** 기사 ************* \n")
    print(rank_li[0])
    print(rank_li[1])
    print(rank_li[2])

    
    new_list = []
    
    for tag in tag_count:
        c = (tag['키워드'], tag['중복수'])
        new_list.append(c)
        
        
    mask1 = np.array(Image.open("cloud.png"))
    font_path = 'C:/Windows/Fonts/malgun.ttf';
    wc = WordCloud(font_path = font_path, background_color = 'white', 
        width=800, height=600, mask = mask1)
    cloud = wc.generate_from_frequencies(dict(new_list))
    plt.figure(figsize=(10,8), dpi=100)
    plt.axis('off')
    plt.imshow(cloud)
    plt.show()
    
    category_index_point += 1
    
    
    print('\n' * 2)
    

f3 = open('total.txt', 'r')

lines = f3.readline()
news_topic_count = 1

line_list.clear()
while lines:
    
    lines = f3.readline()
    news_topic_count += 1


print('전체 기사 수 :{}'.format(news_topic_count))

f3.close()
    

f = open('total.txt', 'r')
lines = f.read()
#### 순위를 찾아내기위한 문장 리스트###
line_list = lines.split('\n')
f.close()

nlpy = Okt()
nouns = nlpy.nouns(lines)
count = Counter(nouns)

total_tag_count = []


for n, c in count.most_common(100):
    dics = {'키워드' : n, '중복수' : c}
    if len(dics['키워드']) >= 2 and len(total_tag_count) <= 100:
        total_tag_count.append(dics)
        
rank=[]


print('---------------카테고리[전체 ]--------------------' )
for idx,tag in enumerate(total_tag_count):
 #### 순위 3위까지의 리스트 뽑아내기 && 기사 뽑아내기#####
    if(idx == 0 or idx == 1 or idx == 2 ):
        rank.append(tag['키워드'])
          
print(pd.DataFrame(total_tag_count[:20], index = list(range(1,21)) ))

print("\n ********** 순위 ************* \n")  
print('\t 1순위:{}'.format(rank[0]))
print('\t 2순위:{}'.format(rank[1]))
print('\t 3순위:{}'.format(rank[2]))  

rank_li.clear() 


temp=0
for line_li in line_list:
    if rank[0] in line_li:
        rank_li.append(line_li+'\n')
        break

temp=0
for line_li in line_list:
    if rank[1] in line_li:
        rank_li.append(line_li+'\n')
        break
        
temp=0
for line_li in line_list:
    if rank[2] in line_li:
        rank_li.append(line_li+'\n')  
        break
              
            
print("\n ********** 기사 ************* \n")
print(rank_li[0])
print(rank_li[1])
print(rank_li[2])

new_list = []

for tag in total_tag_count:
    c = (tag['키워드'], tag['중복수'])
    new_list.append(c)


mask2 = np.array(Image.open("head.png"))
font_path = 'C:/Windows/Fonts/malgun.ttf';
wc = WordCloud(font_path = font_path, background_color = 'white', 
    width=800, height=600, mask = mask2)
cloud = wc.generate_from_frequencies(dict(new_list))
plt.figure(figsize=(10,8), dpi = 100)
plt.axis('off')
plt.imshow(cloud)
plt.show()


AttributeError: 'NoneType' object has no attribute 'find_all'